Motion Generation using Optimization
------------------------------------

Introduction to KOMO:
https://github.com/MarcToussaint/rai/wiki

In [31]:
#import Stuff
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

In [32]:
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")

V = ry.ConfigurationViewer()
V.setConfiguration(C)   #update every time the view

#### Quaternion Info
[w, x, y, z] (?) --> rotation around x, y, z 180 degree [0,1,0,0] [0,0,1,0] [0,0,0,1]
[link for quaternion informations](https://www.euclideanspace.com/maths/geometry/rotations/conversions/eulerToQuaternion/steps/index.htm)


In [33]:
#add object
obj = C.addFrame("object")
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])
obj.setMass(10.0)
V.setConfiguration(C)

## Exersice 1
 a.) Compute a 2-arm robot configuration, where the graspCenter positions of both hands coincide, the two hands oppose, and their x-axes are orthogonal. (E.g., as if they would handover a little cube.)

In [8]:
#red - x entlang gripper
#green - y orthogonal zu gripper
#blue - z inside gripper

#Scalar product
# A dot B = 0 --> 90 degree
# A dot B = 1 --> 0 degree
# A dot B = -1 --> 180 degree

X0 = C.getFrameState()
#print(C.getFrameNames())
#C.frame("R_gripperCenter").setQuaternion([0,0,0,1]) # [w, z, y, x] ????

IK = C.komo_IK(False)
IK.addObjective([], ry.FS.positionDiff, ["R_gripperCenter", "L_gripperCenter"], ry.OT.eq)
IK.addObjective([], ry.FS.scalarProductZZ, ["R_gripperCenter", "L_gripperCenter"], ry.OT.eq, target=[-1])
IK.addObjective([], ry.FS.scalarProductXY, ["R_gripperCenter", "L_gripperCenter"], ry.OT.eq, target=[1])

IK.optimize(True)
IK.view()

#V.setConfiguration(C)

b.) Add a box (shape type ssBox, see Tip2 below) somewhere to the scene, compute a robot configuration where one of the grippers grasps the box (centered, along a particular axis), while avoiding collisions between the box and the two fingers and gripper.

### ssBox and ssCvx

ssBox - sphere-swept box. This is a box with rounded corners. This should be your default primitive shape.
The shape is determined by 4 numbers: **x-size, y-size, z-size, radius of corners**.

The 2nd most important shape type is ssCvx (sphere-swept convex), which is determined by a set of 3D points, and sphere radius that is added to the points’ convex hull.

(E.g., a capsule can also be described as simple ssCvx: 2 points with a sweeping radius.) The sphere-swept shape primitives allow for well-defined Jacobians of collision features.

In [30]:
box = C.addFrame("box")
box.setPosition([.4,0,1])
box.setShape(ry.ST.ssBox, [.1, .1, .25, .02])
box.setColor([0,0,1])
box.setContact(1)

C.setFrameState(X0)     # reset
V.setConfiguration(C)

In [ ]:
IK = C.komo_IK(True)
IK.addObjective([], ry.FS.accumulatedCollisions)

